# OpenAQ PM2.5 Data Analysis - 2023 Dataset

Analysis of PM2.5 air quality data from OpenAQ for 2023.

**Version**: V2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 10
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
data_path = '/Users/vojtech/Code/Bard89/Project-Data/data/processed/jp_openaq_processed_20230101_to_20231231.csv'
print(f"Loading OpenAQ data from: {data_path}")

df = pd.read_csv(data_path)
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
df_2023 = df[(df['timestamp'] >= '2023-01-01') & (df['timestamp'] < '2024-01-01')]
print(f"2023 data: {len(df_2023):,} records")
print(f"2023 date range: {df_2023['timestamp'].min()} to {df_2023['timestamp'].max()}")

## 1. Dataset Overview

In [ ]:
print("Dataset Info:")
print("="*60)
print(f"Total records: {len(df_2023):,}")
print(f"Unique hexagons (res8): {df_2023['h3_index_res8'].nunique():,}")
print(f"Date range: {df_2023['timestamp'].min()} to {df_2023['timestamp'].max()}")
print(f"\nColumns ({len(df_2023.columns)}):")
for col in df_2023.columns:
    print(f"  - {col}: {df_2023[col].dtype}")

In [ ]:
print("First 10 rows:")
display(df_2023.head(10))

print("\nLast 10 rows:")
display(df_2023.tail(10))

## 2. Temporal Coverage Analysis

In [ ]:
df_2023['date'] = df_2023['timestamp'].dt.date
all_dates_2023 = pd.date_range('2023-01-01', '2023-12-31', freq='D').date
existing_dates = set(df_2023['date'].unique())
missing_dates = sorted(set(all_dates_2023) - existing_dates)

print(f"Temporal Coverage Analysis for 2023:")
print("="*60)
print(f"Expected days in 2023: 365")
print(f"Days with data: {len(existing_dates)}")
print(f"Missing days: {len(missing_dates)}")
print(f"Coverage: {len(existing_dates)/365*100:.1f}%")

print(f"\n⚠️ DATA STARTS FROM: {df_2023['timestamp'].min().date()}")
print(f"Missing period: 2023-01-01 to 2023-07-13 ({len(missing_dates)} days)")

print("\nMonthly coverage:")
monthly_counts = df_2023.groupby(df_2023['timestamp'].dt.to_period('M')).size()
all_months = pd.period_range('2023-01', '2023-12', freq='M')
for month in all_months:
    actual_records = monthly_counts.get(month, 0)
    print(f"  {month}: {actual_records:,} records")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

calendar_data = np.zeros((12, 31))
for date in all_dates_2023:
    month = date.month - 1
    day = date.day - 1
    if day < 31:
        calendar_data[month, day] = 1 if date in existing_dates else -1

calendar_data[calendar_data == 0] = np.nan

im = axes[0, 0].imshow(calendar_data, cmap='RdYlGn', aspect='auto', vmin=-1, vmax=1)
axes[0, 0].set_title('2023 OpenAQ Data Availability Calendar', fontsize=12)
axes[0, 0].set_xlabel('Day of Month')
axes[0, 0].set_ylabel('Month')
axes[0, 0].set_yticks(range(12))
axes[0, 0].set_yticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
axes[0, 0].set_xticks(range(0, 31, 5))
axes[0, 0].set_xticklabels(range(1, 32, 5))
axes[0, 0].axhline(y=6.5, color='red', linewidth=2, linestyle='--', label='Data starts')
plt.colorbar(im, ax=axes[0, 0], label='Data Available')

date_range = pd.date_range('2023-01-01', '2023-12-31', freq='D')
daily_counts = df_2023.groupby('date').size().reindex(date_range.date, fill_value=0)
axes[0, 1].plot(daily_counts.index, daily_counts.values, linewidth=1)
axes[0, 1].set_title('Daily Record Count Throughout 2023', fontsize=12)
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Number of Records')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].set_xlim(date_range[0], date_range[-1])
axes[0, 1].axvspan(pd.Timestamp('2023-01-01'), pd.Timestamp('2023-07-13'),
                   color='red', alpha=0.2, label='Missing period')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

monthly_coverage = df_2023.groupby(df_2023['timestamp'].dt.to_period('M')).size().reindex(all_months, fill_value=0)
colors = ['red' if i < 6 else 'orange' if i == 6 else 'green' for i in range(12)]
bars = axes[1, 0].bar(range(12), monthly_coverage.values, color=colors, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Monthly Record Count for 2023', fontsize=12)
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Number of Records')
axes[1, 0].set_xticks(range(12))
axes[1, 0].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                             'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
axes[1, 0].grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, monthly_coverage.values):
    axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5000,
                    f'{val:,}' if val > 0 else 'NO DATA', ha='center', va='bottom', fontsize=8)

hourly_counts = df_2023.groupby(df_2023['timestamp'].dt.hour).size()
axes[1, 1].bar(hourly_counts.index, hourly_counts.values, color='steelblue', edgecolor='black')
axes[1, 1].set_title('Hourly Distribution of Records', fontsize=12)
axes[1, 1].set_xlabel('Hour of Day')
axes[1, 1].set_ylabel('Number of Records')
axes[1, 1].set_xticks(range(0, 24, 2))
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Temporal Coverage Analysis - OpenAQ 2023', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 3. PM2.5 Data Analysis

In [ ]:
pm25_columns = [col for col in df_2023.columns if 'pm25' in col.lower()]
print(f"PM2.5 columns found: {pm25_columns}")

missing_stats = pd.DataFrame({
    'Missing Count': df_2023[pm25_columns].isnull().sum(),
    'Missing %': (df_2023[pm25_columns].isnull().sum() / len(df_2023) * 100).round(2),
    'Available Count': df_2023[pm25_columns].notnull().sum(),
    'Available %': (df_2023[pm25_columns].notnull().sum() / len(df_2023) * 100).round(2)
})

print("\nPM2.5 Data Completeness:")
print("="*60)
display(missing_stats)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

pm25_mean = df_2023['pm25_ugm3_mean'].dropna()
axes[0, 0].hist(pm25_mean[pm25_mean <= 100], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('PM2.5 (μg/m³)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('PM2.5 Distribution (≤100 μg/m³)')
axes[0, 0].axvline(pm25_mean.mean(), color='red', linestyle='--', label=f'Mean: {pm25_mean.mean():.1f}')
axes[0, 0].axvline(pm25_mean.median(), color='green', linestyle='--', label=f'Median: {pm25_mean.median():.1f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

daily_pm25_missing = df_2023.groupby('date')['pm25_ugm3_mean'].apply(lambda x: x.isnull().mean() * 100)
axes[0, 1].plot(daily_pm25_missing.index, daily_pm25_missing.values, linewidth=1)
axes[0, 1].set_title('PM2.5 Missing Data Over Time', fontsize=12)
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Missing %')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

monthly_pm25 = df_2023.groupby(df_2023['timestamp'].dt.month)['pm25_ugm3_mean'].mean()
available_months = monthly_pm25.index
axes[1, 0].bar(available_months, monthly_pm25.values, color='steelblue', edgecolor='black')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average PM2.5 (μg/m³)')
axes[1, 0].set_title('Average PM2.5 by Month (Available Months Only)')
axes[1, 0].set_xticks(available_months)
axes[1, 0].set_xticklabels([['', '', '', '', '', '', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][m-1] for m in available_months])
axes[1, 0].grid(True, alpha=0.3, axis='y')

hourly_pm25 = df_2023.groupby(df_2023['timestamp'].dt.hour)['pm25_ugm3_mean'].mean()
axes[1, 1].plot(hourly_pm25.index, hourly_pm25.values, marker='o', linewidth=2)
axes[1, 1].set_xlabel('Hour of Day')
axes[1, 1].set_ylabel('Average PM2.5 (μg/m³)')
axes[1, 1].set_title('Average PM2.5 by Hour of Day')
axes[1, 1].set_xticks(range(0, 24, 2))
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('PM2.5 Data Analysis - OpenAQ 2023', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 4. Geographic Coverage

In [ ]:
hex_locations = df_2023[['h3_index_res8', 'h3_lat_res8', 'h3_lon_res8']].drop_duplicates()
print(f"Geographic Coverage:")
print("="*60)
print(f"Unique hexagons: {len(hex_locations):,}")
print(f"Latitude range: {hex_locations['h3_lat_res8'].min():.2f} to {hex_locations['h3_lat_res8'].max():.2f}")
print(f"Longitude range: {hex_locations['h3_lon_res8'].min():.2f} to {hex_locations['h3_lon_res8'].max():.2f}")

hex_data_counts = df_2023.groupby('h3_index_res8').agg({
    'pm25_ugm3_mean': ['count', 'mean', lambda x: x.notna().mean()]
}).reset_index()
hex_data_counts.columns = ['h3_index_res8', 'record_count', 'pm25_mean', 'pm25_coverage']
hex_with_counts = hex_locations.merge(hex_data_counts, on='h3_index_res8')

print(f"\nRecords per hexagon:")
print(f"  Mean: {hex_with_counts['record_count'].mean():.0f}")
print(f"  Median: {hex_with_counts['record_count'].median():.0f}")
print(f"  Min: {hex_with_counts['record_count'].min()}")
print(f"  Max: {hex_with_counts['record_count'].max()}")

print(f"\nPM2.5 data completeness per hexagon:")
print(f"  Mean: {hex_with_counts['pm25_coverage'].mean():.1%}")
print(f"  Median: {hex_with_counts['pm25_coverage'].median():.1%}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

scatter = axes[0].scatter(hex_with_counts['h3_lon_res8'],
                         hex_with_counts['h3_lat_res8'],
                         c=hex_with_counts['pm25_mean'],
                         cmap='YlOrRd', s=20, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('PM2.5 Monitoring Locations with Average PM2.5 Levels')
plt.colorbar(scatter, ax=axes[0], label='Mean PM2.5 (μg/m³)')

scatter2 = axes[1].scatter(hex_with_counts['h3_lon_res8'],
                          hex_with_counts['h3_lat_res8'],
                          c=hex_with_counts['pm25_coverage']*100,
                          cmap='viridis', s=20, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_title('PM2.5 Data Coverage by Location')
plt.colorbar(scatter2, ax=axes[1], label='Data Coverage (%)')

plt.tight_layout()
plt.show()

## 5. Statistical Summary

In [ ]:
print("PM2.5 Statistical Summary:")
print("="*60)
display(df_2023[pm25_columns].describe())

In [ ]:
print("OPENAQ 2023 DATA QUALITY SUMMARY")
print("="*60)

print("\n📊 DATASET OVERVIEW:")
print(f"   Total records: {len(df_2023):,}")
print(f"   Time period: {df_2023['timestamp'].min().date()} to {df_2023['timestamp'].max().date()}")
print(f"   Unique locations (hexagons): {df_2023['h3_index_res8'].nunique()}")
print(f"   Temporal resolution: Hourly")

print("\n⚠️ TEMPORAL COVERAGE:")
print(f"   Days with data: {len(existing_dates)}/365 ({len(existing_dates)/365*100:.1f}%)")
print(f"   ❌ MISSING: January 1 - July 13, 2023 (194 days)")
print(f"   ✓ Available: July 14 - December 31, 2023 (171 days)")

print("\n📈 PM2.5 STATISTICS (Available Period):")
print(f"   Mean: {pm25_mean.mean():.2f} μg/m³")
print(f"   Median: {pm25_mean.median():.2f} μg/m³")
print(f"   Std Dev: {pm25_mean.std():.2f} μg/m³")
print(f"   Min: {pm25_mean.min():.2f} μg/m³")
print(f"   Max: {pm25_mean.max():.2f} μg/m³")
print(f"   95th percentile: {pm25_mean.quantile(0.95):.2f} μg/m³")

print("\n✅ DATA COMPLETENESS (Within Available Period):")
for col in pm25_columns:
    completeness = df_2023[col].notna().mean() * 100
    print(f"   {col}: {completeness:.1f}%")

print("\n🔴 CRITICAL FINDING:")
print("   The OpenAQ dataset is missing the first half of 2023 (Jan-Jul 13).")
print("   This explains why the enriched PM2.5 dataset only contains data")
print("   from July 14, 2023 onwards, despite having complete weather and")
print("   traffic data for the entire year.")